### Deploy and interact with LUCE contract
The aim of this notebook is to compile, deploy and interact with all functions of the LUCE smart contract (v3).

### Compile contract from file

In [1]:
# Import libraries
import json
import web3

from web3 import Web3
from solcx import compile_source
from web3.contract import ConciseContract

Using solc version v0.4.25


In [11]:
# Read in LUCE contract code
with open('./data/luce.sol', 'r') as file:
    contract_source_code = file.read()

In [12]:
# Compile
compiled_sol = compile_source(contract_source_code) # Compiled source code

In [ ]:
#compiled_sol

In [13]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Dataset']

In [ ]:
#contract_interface

In [14]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [15]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:8545"))

In [16]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

The default account address is used as the default "from" property in other functions if no other explicit "from" property is specified.

In [20]:
# Create contract blueprint instance
Luce = w3.eth.contract(abi=abi, bytecode=bytecode)

In [21]:
# Submit the transaction that deploys the contract
tx_hash = Dataset_contract.constructor().transact()

### Obtain Transcation Receipt

In [22]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [23]:
tx_receipt.contractAddress

'0x7C6F2118D40f17631F08e93818bF895FF9b275E3'

### Interact with contract

In [24]:
# Create instance of deployed contract
luce1 = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [34]:
# Publish Data
tx_hash = luce1.functions.publishData("my description", "my link", 6).transact()

In [35]:
# Get Licence
luce1.functions.getLicence().call()

6

In [39]:
# Set Licence
_ = luce1.functions.setLicence(28).transact()
luce1.functions.getLicence().call()

28

I am using `_` to suppress the output of the transaction hash. I do this since transactions are mined instantaneously in the local ganache test setting. On the testnet and mainnet we would store the transaction hash, then wait until a transaction receipt is available and only then we can be sure that the transaction went through.

In [43]:
# Update Description
_ = luce1.functions.updateData("New Description", "New Link").transact()

In [49]:
luce1.functions.getLicence().call()

28

In [51]:
luce1.functions.getLink(1).call()

'New Link'

In [52]:
luce1.functions.dataDescription().call()

'New Description'

In [54]:
luce1.functions.dataProvider().call()

'0x92D44e8579620F2Db88A12E70FE38e8CDB3541BA'

Public variables in smart contracts automatically have a getter method. Since the Data Provider and Data Description are public they can simply be called.

In [61]:
#luce1.functions.addDataRequester(1,2).transact()

This method throws an error at the moment

In [62]:
#luce1.functions.renewToken(1).transact()
# Error: Need to agree on licence first

Look at last two methods in LUCE contract and try to fix. Everything else is working.